# Trajectory Follower Kinematic Simulation Sandbox

Self-contained example of `home_robot.control.traj_following_controller.TrajFollower` in action.

- The controller accepts a "trajectory" in the form of a function: `xyt, dxyt, done = traj(t)`.
- The controller stabilizes around the trajectory, instead of moving directly to the goal location like the GotoVelocityController.
- The trajectory can be updated mid-execution.

### Requirements
Pip
- numpy
- scipy
- matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from home_robot.control.traj_following_controller import TrajFollower

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150

In [ ]:
# ----- HYPERPARAMS
# Simulation params
SPEEDUP = 4

TIME_HORIZON = 12
SIM_HZ = 60
CONTROL_HZ = 20
VIZ_HZ = 24 / SPEEDUP

# Visualization params
R_ROBOT = 0.2
BUTT_SAMPLES = 20
X_LIM = [-2, 2]
Y_LIM = [-2, 2]

In [ ]:
# ----- SIMULATION ENV
class Env:
    """Simple kinematic sim of a diff drive robot"""
    
    def __init__(self, hz, sys_noise=None):
        self.pos_state = np.zeros(3)  # x, y, t
        self.vel_state = np.zeros(2)  # v, w
        self.dt = 1.0 / hz
        
        self.noise = np.zeros(3) if sys_noise is None else np.array(sys_noise)
        
    def get_pose(self):
        return self.pos_state
    
    def step(self, vel_input=None):
        if vel_input is not None:
            self.vel_state = vel_input
            
        self.pos_state[0] += self.vel_state[0] * np.cos(self.pos_state[2]) * self.dt
        self.pos_state[1] += self.vel_state[0] * np.sin(self.pos_state[2]) * self.dt
        self.pos_state[2] += self.vel_state[1] * self.dt
        self.pos_state += self.noise * np.random.randn(3) * self.dt

In [ ]:
# ----- VISUALIZER
class Viz:
    """Visualize and animate robot states"""

    def __init__(self, title=""):
        self.title = title
        self.frames = []
        
        # Robot local vertices
        xy_ul = np.array([R_ROBOT / 2, R_ROBOT])
        xy_ur = np.array([R_ROBOT / 2, -R_ROBOT])
        xy_l = np.array([-R_ROBOT / 2, R_ROBOT])
        xy_r = np.array([-R_ROBOT / 2, -R_ROBOT])
        
        xy_list = [xy_r, xy_ur, xy_ul, xy_l]
        for phi in np.linspace(np.pi / 2, 3 * np.pi / 2, BUTT_SAMPLES):
            xy_list.append(np.array([R_ROBOT * (np.cos(phi) - 0.5), R_ROBOT * np.sin(phi)]))
            
        self.xy_verts_local = np.array(xy_list)
        
        # Trajectory
        traj_path = None
        
    def _draw_robot(self, xyt, color):
        theta = xyt[2]
        ct = np.cos(theta)
        st = np.sin(theta)
        trans = np.array([[ct, -st], [st, ct]])
        
        xy_verts_transformed = (trans @ self.xy_verts_local.T).T + xyt[None, :2]
        
        self.ax.plot(xyt[0], xyt[1], color=color, marker='o')
        self.ax.plot(xy_verts_transformed[:, 0], xy_verts_transformed[:, 1], color=color)
        
    def _draw_traj(self, traj_path, color):
        self.ax.plot(traj_path[:, 0], traj_path[:, 1], color=color)
        
    def _render_frame(self, frame):
        plt.cla()
        
        self.ax.set_xlim(*X_LIM)
        self.ax.set_ylim(*Y_LIM)
        self.ax.set_aspect("equal")
        
        if self.traj_path is not None:
            self._draw_traj(self.traj_path, '0.5')
        self._draw_robot(frame[1], '0.7')
        self._draw_robot(frame[0], '0')
        
    def clear_frames(self):
        self.frames = []
        
    def set_trajectory(self, traj_func, T, N):
        """Assign trajectory at beginning of episode to visualize desired states"""
        self.traj_path = np.zeros([N, 2])
        for i in range(N):
            t = i * T / N
            xyt, _, _ = traj_func(t)
            self.traj_path[i, :] = xyt[:2]
    
    def record_frame(self, xyt_robot, xyt_goal):
        assert xyt_robot.shape[-1] == 3
        assert xyt_goal.shape[-1] == 3
        self.frames.append((xyt_robot, xyt_goal))
        
    def animate(self, playback_speed=1.0):
        """Animate all recorded frames"""
        plt.ioff()
        fig, self.ax = plt.subplots()
        self.ax.set_title(self.title)
        
        return animation.FuncAnimation(fig, self._render_frame, frames=self.frames, interval=1000 / VIZ_HZ / playback_speed, repeat=False)

In [ ]:
# ----- GENERATE TRAJ: Circle
R = 1.

w = np.pi / 2 / TIME_HORIZON

def traj_func(t):
    """A trajectory function that returns the desired SE2 position & velocity given a timestamp"""
    t_scaled = t / TIME_HORIZON
    theta = t_scaled * (np.pi / 2)
    
    xyt =  np.array([
        R * np.sin(theta),
        R * (1. - np.cos(theta)),
        theta
    ])
    dxyt = np.array([
        R * w * np.cos(theta),
        R * w * np.sin(theta),
        w
    ])
    done = t >= TIME_HORIZON
    
    return xyt, dxyt, done

In [ ]:
# ----- CONTROLLER
from omegaconf import OmegaConf

def create_controller(
    v_max=0.2,
    w_max=0.45,
    k_p=0.5,
    damp_ratio=1.0,
    decay=0.5,
):
    # Generate config
    cfg_dict = {
        "v_max": v_max,
        "w_max": w_max,
        "k_p": k_p,
        "damp_ratio": damp_ratio,
        "decay": decay,
    }
    cfg = OmegaConf.create(cfg_dict)

    # Instatiate controller
    return TrajFollower(cfg)  # default config "traj_follower.yaml" will be loaded if no cfg is provided

In [ ]:
# ----- MAIN
OPEN_LOOP = False  # change this value to compare using the TrajFollower vs. open-loop execution of the trajectory

# initialize objects
xyt_goal, _, _ = traj_func(TIME_HORIZON)

viz = Viz()
env = Env(SIM_HZ, sys_noise=[0.03, 0.03, 0.2])  # inject noise into system to observe closed-loop control behavior
controller = create_controller()

# set traj
controller.update_trajectory(traj_func)  # Assign trajectory to controller
viz.set_trajectory(traj_func, TIME_HORIZON, int(TIME_HORIZON * SIM_HZ))

# simulate
t_control = 0
t_viz = 0
dt_control = 1 / CONTROL_HZ
dt_viz = 1 / VIZ_HZ
for t in np.linspace(0, TIME_HORIZON, int(TIME_HORIZON * SIM_HZ)):
    xyt = env.get_pose()

    vel_command = None
    
    # Query controller at a frequency of CONTROL_HZ
    if t >= t_control:
        # Open loop control: directly apply desired velocities
        if OPEN_LOOP:
            _, dxyt_goal, _ = traj_func(t)
            vel_command = [np.linalg.norm(dxyt_goal[:2]), dxyt_goal[2]]
    
        # Closed loop control: Use traj follower
        else:
            vel_command = controller.forward(xyt, t)
        t_control += dt_control
        
    # Record frames at a frequency of VIZ_HZ
    if t >= t_viz:
        xyt_goal, _, _ = traj_func(t)
        viz.record_frame(xyt.copy(), xyt_goal.copy())
        t_viz += dt_viz

    env.step(vel_command)

In [ ]:
# animate
viz.animate(playback_speed=SPEEDUP)